# References
https://pyomo.readthedocs.io/en/stable/pyomo_overview/simple_examples.html

```
conda install -c conda-forge pyomo
```

In [3]:
import pyomo.environ as pyo
import numpy as np

In this tutorial, I'll use the Gurobi solver. Students and academics can register for a free license [here](https://www.gurobi.com/academia/academic-program-and-licenses/). 

Pyomo also suports several other solvers. The solvers available for your version of pyomo can be listed by running the following command in an anaconda prompt using an environment with pyomo installed.
```
pyomo help --solvers
```

# Minimal Example

$Maximize\left( 40 \ x \right)$

s.t. 

$x \leq 40$

In [7]:
model = pyo.ConcreteModel()

model.x = pyo.Var(domain=pyo.NonNegativeReals)

model.objFunc = pyo.Objective(
    expr = 40 * model.x,
    sense = pyo.maximize
)

model.demandConstraint = pyo.Constraint(expr = model.x <= 40)

pyo.SolverFactory('gurobi').solve(model)#.write()

#model.objFunc.display()
print("ObjFunc = {}".format(model.objFunc()))
print("x = {}".format(model.x()))

ObjFunc = 1600.0
x = 40.0


# Array of Variables Example

$Maximize\left(\sum_{i \in I} c_i x_i \right)$

s.t. 

$x_i \leq b_i \ \ \ \ \forall \ i \in I$

In [10]:
model = pyo.ConcreteModel()

numVar = 10

model.I = pyo.Set(initialize=range(numVar))

model.x = pyo.Var(model.I, domain=pyo.NonNegativeReals)

c = np.random.uniform(0,1,numVar)
b = np.random.uniform(0,1,numVar)

model.objFunc = pyo.Objective(
    expr = sum([c[i] * model.x[i] for i in model.I]),
    sense = pyo.maximize
)

def myConstraint(m,i):
    return m.x[i] <= b[i]
model.myConstraint = pyo.Constraint(model.I, rule=myConstraint)

pyo.SolverFactory('gurobi').solve(model)#.write()

#model.objFunc.display()
print("ObjFunc = {}".format(model.objFunc()))
model.x.display()

ObjFunc = 2.517802331273474
x : Size=10, Index=I
    Key : Lower : Value               : Upper : Fixed : Stale : Domain
      0 :     0 : 0.37690870596694015 :  None : False : False : NonNegativeReals
      1 :     0 :  0.3257149920412692 :  None : False : False : NonNegativeReals
      2 :     0 :  0.5773276138039757 :  None : False : False : NonNegativeReals
      3 :     0 :  0.9838212872568641 :  None : False : False : NonNegativeReals
      4 :     0 :  0.8636205814788136 :  None : False : False : NonNegativeReals
      5 :     0 :  0.7417648603245927 :  None : False : False : NonNegativeReals
      6 :     0 :  0.8138812528075818 :  None : False : False : NonNegativeReals
      7 :     0 : 0.44816866002037126 :  None : False : False : NonNegativeReals
      8 :     0 :  0.8896687815465013 :  None : False : False : NonNegativeReals
      9 :     0 : 0.38861722190339176 :  None : False : False : NonNegativeReals


# Multi-Dimensional Array of Variables Example

$Maximize\left(\sum_{i \in I} \sum_{j \in J} \sum_{k \in K} c_{i,j,k} x_{i,j,k} \right)$

s.t. 

$\sum_{k \in K} x_{i,j,k} \leq b_{i,j} \ \ \ \ \forall \ i \in \ I, \ j \ \in \ J$

In [13]:
model = pyo.ConcreteModel()

numI = 3
numJ = 4
numK = 5

model.I = pyo.Set(initialize=range(numI))
model.J = pyo.Set(initialize=range(numJ))
model.K = pyo.Set(initialize=range(numK))

model.I_J = pyo.Set(initialize=model.I * model.J)
model.I_J_K = pyo.Set(initialize=model.I * model.J * model.K)

model.x = pyo.Var(model.I_J_K, domain=pyo.NonNegativeReals)

c = np.random.uniform(0,1,(numI,numJ,numK))
b = np.random.uniform(0,1,(numI,numJ))

model.objFunc = pyo.Objective(
    expr = sum([c[i,j,k] * model.x[i,j,k] for i in model.I for j in model.J for k in model.K]),
    sense = pyo.maximize
)

def myConstraint(m,i,j):
    return sum([m.x[i,j,k] for k in m.K]) <= b[i,j]
model.myConstraint = pyo.Constraint(model.I_J, rule=myConstraint)

pyo.SolverFactory('gurobi').solve(model)#.write()

#model.objFunc.display()
print("ObjFunc = {}".format(model.objFunc()))
model.x.display()

ObjFunc = 3.5353518712345475
x : Size=60, Index=I_J_K
    Key       : Lower : Value               : Upper : Fixed : Stale : Domain
    (0, 0, 0) :     0 :                 0.0 :  None : False : False : NonNegativeReals
    (0, 0, 1) :     0 :                 0.0 :  None : False : False : NonNegativeReals
    (0, 0, 2) :     0 :                 0.0 :  None : False : False : NonNegativeReals
    (0, 0, 3) :     0 :                 0.0 :  None : False : False : NonNegativeReals
    (0, 0, 4) :     0 : 0.29518878210040833 :  None : False : False : NonNegativeReals
    (0, 1, 0) :     0 :                 0.0 :  None : False : False : NonNegativeReals
    (0, 1, 1) :     0 :                 0.0 :  None : False : False : NonNegativeReals
    (0, 1, 2) :     0 :                 0.0 :  None : False : False : NonNegativeReals
    (0, 1, 3) :     0 :                 0.0 :  None : False : False : NonNegativeReals
    (0, 1, 4) :     0 :  0.3033402779612966 :  None : False : False : NonNegativeReals